# Model Processing

## Imports & General Settings 

In [90]:
import unittest
import os
import sys
import time
import pathlib

import matplotlib as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm

# Our imports
from data import WaveletTransform, AFECGDataset
import dsp
from model.blocks import ConvNet, BRNN, SoftmaxAttention
from model.baseline import Baseline
from training import train


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [6]:
dataset_name = 'afdb'
dataset = AFECGDataset(dataset_name, '../data/files/')

In [7]:
dataset.load()

25it [00:00, 249.40it/s]

Preparing 1397 samples


1397it [00:04, 336.28it/s]


Elapsed time: 4165.770053863525 ms
Skipped 1397 files which had a backup


In [8]:
data, label = dataset[0]
data.shape

torch.Size([20, 20, 375])

In [9]:
images_per_sample = 20
total_data_size = len(dataset)
print("Total data size: ", total_data_size)

Total data size:  1397


In [10]:
# data = [dataset[i][0] for i in range(total_data_size)]
# labels = [dataset[i][1] for i in range(total_data_size)]

### Example of one ECG sample

In [11]:
# samples, label = data[0], labels[0]
# print('P-signal: ', samples)
# print('Has AF: ', 'Yes' if label == 1 else 'No')

In [12]:
# to_wavelet = WaveletTransform(wavelet.Morlet(6), resample=20)
# t = to_wavelet(data[0][0])
# image_test = (t * 100 * 255).int() # Simple visualization test
# transforms.ToPILImage()(image_test).show()

##  Wavelet Transform

In [13]:
# Total data size is 1397
# You can choose the data size 
data_size = len(dataset)

## Train & Test set creation

## CNN

In [15]:
display(ConvNet((256, 256)))

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 21), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(10, 10, kernel_size=(4, 21), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Linear(in_features=2540, out_features=50, bias=True)
)

In [16]:
# x0 = x_train[0][0].float()
# encoder_cnn = ConvNet((375, 20))

# display(x0.unsqueeze(0).shape)
# h = encoder_cnn(x0.unsqueeze(0))
# print(h.shape)

# test.assertEqual(h.dim(), 2)
# test.assertSequenceEqual(h.shape, (1, 50))

## BRNN

In [17]:
display(BRNN(50, 50, images_per_sample))

BRNN(
  (bi_rnn): RNN(50, 50, num_layers=20, bidirectional=True)
)

## Attention

Notations:

* $Y = \left[ y_1, \ldots, y_T \right]$ – the input matrix of size $\left( N \times T \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $w_\mathrm{att}$ – The parameters of the attention model, of size $\left( N \times 1 \right)$, where $N$ is the number of features in a single output vector of the BRNN

* $\alpha$ – The attention weights, given as $\alpha = \mathrm{softmax} \left( w_\mathrm{att}^T Y \right)$. This is an element-wise softmax, where the output size of $\alpha$ is $\left( 1 \times T \right)$

* $h_\mathrm{att}$ – Output of the attention mechanism, given by $h_\mathrm{att} = Y \alpha^T$, of size $\left( N \times 1 \right)$, i.e. a vector of $N$ features.

In [25]:
SoftmaxAttention(100)

SoftmaxAttention()

## Full model

In [93]:
model = Baseline()

### Training 

In [94]:
data1, label1 = dataset[0]
data2, label2 = dataset[1]
batch_data = torch.cat([data1.unsqueeze(0) * 10, data2.unsqueeze(0) * 10], dim=0)
print(batch_data.shape)
output = model(batch_data)

torch.Size([2, 20, 20, 375])
After FC:  tensor([[-6.1794,  8.4013],
        [-6.1790,  8.4008]], grad_fn=<SliceBackward>)


In [97]:
model = Baseline()
config = dict(
    num_workers=4,
    batch_size=90,
    learning_rate=0.001,
    weight_decay=0.01,
    num_epochs=200,
    is_notebook=True
)

train(model, dataset, config)

After FC:  tensor([[ 0.1204, -0.0548],
        [ 0.1146, -0.0491],
        [ 0.1159, -0.0518],
        [ 0.1210, -0.0560],
        [ 0.1198, -0.0543],
        [ 0.1210, -0.0531],
        [ 0.1186, -0.0534],
        [ 0.1128, -0.0527],
        [ 0.1147, -0.0526],
        [ 0.1184, -0.0510]], grad_fn=<SliceBackward>)
tensor([[0.5437, 0.4563],
        [0.5408, 0.4592],
        [0.5418, 0.4582],
        [0.5441, 0.4559],
        [0.5434, 0.4566],
        [0.5434, 0.4566],
        [0.5429, 0.4571],
        [0.5413, 0.4587],
        [0.5417, 0.4583],
        [0.5423, 0.4577]], grad_fn=<SliceBackward>)
Labels: tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1])
Output: tensor([[0.5437, 0.4563],
        [0.5408, 0.4592],
        [0.5418, 0.4582],


Output: tensor([[0.5914, 0.4086],
        [0.5915, 0.4085],
        [0.5915, 0.4085],
        [0.5913, 0.4087],
        [0.5915, 0.4085],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5913, 0.4087],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5915, 0.4085],
        [0.5913, 0.4087],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5915, 0.4085],
        [0.5915, 0.4085],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5915, 0.4085],
        [0.5914, 0.4086],
        [0.5915, 0.4085],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5915, 0.4085],
        [0.5914, 0.4086],
        [0.5914, 0.4086],
        [0.5915, 0.4085],
    

After FC:  tensor([[ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715],
        [ 0.3657, -0.5715]], grad_fn=<SliceBackward>)
tensor([[0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815],
        [0.7185, 0.2815]], grad_fn=<SliceBackward>)
Labels: tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0])




Traceback (most recent call last):
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/miki/opt/miniconda3/envs/cs236781-project/lib/python3.7/multiprocessing/connect

KeyboardInterrupt: 